In [1]:
import mne
import numpy as np
import os
from glob import glob

from mne_bids import BIDSPath, write_raw_bids

In [2]:
# project directory
proj_dir = os.path.join('/Users/dsj3886/',
                        'data_local',
                        #'EAM1',
                        'EAM1_local',
                        )

# raw data directory (need to specify for your computer)
data_dir = os.path.join(proj_dir, 'EEG_raw')

In [3]:
# output BIDS directory to be created/filled
bids_dir = os.path.join(proj_dir,
                        'data-bids')

In [ ]:
# define function to generate event labels and codes
# (differs per participant)
# TODO: add motor condition
# TODO: fix sub < 13 task-active codes
def generate_event_dict(sub_label, task_label):
    if int(sub_label) > 19:
        event_dict = {'passive/pos': 1,
                    'passive/neg': 2,
                    'active/pos': 3,
                    'active/neg': 4}
    elif int(sub_label) > 13:
        event_dict = {f'{task_label}/pos': 1,
                    f'{task_label}/neg': 2,}
    elif int(sub_label) < 13:
        if task_label == 'passive':
            event_dict = {f'{task_label}/pos': 2049,
                        f'{task_label}/neg': 2050,}
        elif task_label == 'active':
            print('all active coded as 2049.',
                  '  need to fix')
            pass
    return event_dict

In [5]:
# define input EEG file
# TODO: change to loops
sub_label  = '03'
task_label = 'active' # 'passive', 'active'
run_label  = '1'

data_fpath = os.path.join(data_dir,
                          f'sub-{sub_label}_task-{task_label}_run-{run_label}.bdf')

# read in input EEG file
raw_data = mne.io.read_raw_bdf(data_fpath)

Extracting EDF parameters from /Users/dsj3886/data_local/EAM1_local/EEG_raw/sub-03_task-active_run-1.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...


In [ ]:
raw_data

<RawEDF | sub-03_task-active_run-1.bdf, 5 x 13221888 (807.0 s), ~9 kB, data not loaded>

In [ ]:
# find events in the `Status` channel of the raw data
events_raw = mne.find_events(raw_data, 
                             stim_channel='Status', 
                             initial_event=True)

4823 events found on stim channel Status
Event IDs: [ 2048  2049  2056  2177  2303 10240 18432 34816 34817 34818 34824 51200
 51207 67584]


In [ ]:
# explore the data
events_raw[events_raw[:,2]==2049]

array([[  775390,     2048,     2049],
       [  996300,     2048,     2049],
       [ 1006129,     2048,     2049],
       ...,
       [13053225,     2048,     2049],
       [13062510,     2048,     2049],
       [13071252,     2048,     2049]])

In [ ]:
# check the number of events
events_raw[events_raw[:,2]==2049].shape

(1196, 3)

In [ ]:
# Modify event codes based on run type
if int(sub_label) < 13:
    if task_label=='active':
        # change odd-indexed sound events to code 2050
        odd_events = [] # TODO - array of odd indices
        events_raw[events_raw[:,2]==2049][odd_events] = 2050

In [ ]:
# run the function to generate events
event_dict = generate_event_dict(sub_label, task_label)

In [ ]:
# only keep task-related events (currently active and passive runs only)
events = mne.pick_events(events_raw, include=[1, 2, 3, 4, 2049, 2050])
print('task event codes:', np.unique(events[:,2]))

task event codes: [3 4]


In [79]:
# define the BIDSPath object for converting this EEG file
bids_path = BIDSPath(subject=sub_label, 
                     run=run_label, 
                     task=task_label,
                     datatype='eeg', 
                     root=bids_dir, )

In [80]:
# finally, do the raw-to-BIDS conversion
write_raw_bids(raw_data, 
               bids_path=bids_path,   
               events=events,
               event_id=event_dict,
               overwrite=True
              )

Extracting EDF parameters from /Users/dsj3886/data_local/EAM1/EEG_raw/sub-21_task-active_run-4.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Writing '/Users/dsj3886/data_local/EAM1/data-bids/participants.tsv'...
Writing '/Users/dsj3886/data_local/EAM1/data-bids/participants.json'...
Used Annotations descriptions: ['active/neg', 'active/pos']
Writing '/Users/dsj3886/data_local/EAM1/data-bids/sub-21/eeg/sub-21_task-active_run-4_events.tsv'...
Writing '/Users/dsj3886/data_local/EAM1/data-bids/sub-21/eeg/sub-21_task-active_run-4_events.json'...
Writing '/Users/dsj3886/data_local/EAM1/data-bids/dataset_description.json'...
Writing '/Users/dsj3886/data_local/EAM1/data-bids/sub-21/eeg/sub-21_task-active_run-4_eeg.json'...
Writing '/Users/dsj3886/data_local/EAM1/data-bids/sub-21/eeg/sub-21_task-active_run-4_channels.tsv'...
Copying data files to sub-21_task-active_run-4_eeg.bdf
Writing '/Users/dsj3886/data_local/EAM1/data-bids/sub-21/sub-21_scans.tsv

BIDSPath(
root: /Users/dsj3886/data_local/EAM1/data-bids
datatype: eeg
basename: sub-21_task-active_run-4_eeg.bdf)